In [14]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [15]:
!ls /data2/yt8m/code/yt8m

ls: cannot access '/data2/yt8m/code/yt8m': No such file or directory


In [3]:
# p = np.load('/data2/yt8m/code/yt8m/probs_rnn3_tr_0919_1943.npy', allow_pickle=True) lb0.5xx

In [52]:
p1 = np.load('/data2/yt8m/code/yt8m/softmax/rnn3_ft_1005_5492_lb746.npy', allow_pickle=True)

In [53]:
p2 = np.load('/data2/yt8m/code/yt8m/softmax/rnn4_ft_1006_5625.npy', allow_pickle=True)
p1[:2]

array([[9.8455113e-01, 2.0337227e-07, 3.5731500e-07, ..., 6.0892003e-06,
        1.9596441e-06, 4.1937537e-07],
       [9.9528128e-01, 5.2664177e-06, 1.0759398e-06, ..., 1.2880225e-06,
        1.9060346e-05, 3.0180809e-06]], dtype=float32)

In [54]:
p = np.average([p1, p2], 0, weights=[0.5, 0.5])

In [28]:
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn6_mt_1004_5838.npy', allow_pickle=True)
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn3_mt_1004_5910.npy', allow_pickle=True)
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn3_ft_1005_5492.npy', allow_pickle=True)
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn4_ft_1006_5625.npy', allow_pickle=True)
#p = np.load('/data2/yt8m/code/yt8m/softmax/rnn6_ft_1006_5385.npy', allow_pickle=True)
#p = np.load('/mnt/chicm/yt8m/code/yt8m/softmax/transformer_3_1010_3490.npy', allow_pickle=True)
p = np.load('/mnt/chicm/yt8m/code/yt8m/softmax/rnn3_1011_8531.npy', allow_pickle=True)



In [29]:
type(p)

numpy.ndarray

In [30]:
p[:2]

array([[9.9983799e-01, 2.5202105e-09, 2.6833682e-10, ..., 5.5328801e-08,
        1.2534814e-07, 8.5668944e-08],
       [9.9976236e-01, 1.2056420e-08, 4.5830886e-09, ..., 5.3990137e-09,
        3.3379795e-06, 1.6633061e-07]], dtype=float32)

In [31]:
df_test = pd.read_csv('/mnt/chicm/yt8m/3/frame/test_ids.csv')
df_test.head()

,vid,start_time,seg_id
0,MeVj,0,MeVj:0
1,MeVj,5,MeVj:5
2,MeVj,10,MeVj:10
3,MeVj,15,MeVj:15
4,MeVj,20,MeVj:20


In [32]:
def get_classes():
    df = pd.read_csv('/mnt/chicm/yt8m/3/frame/classes_1001.csv')
    c = df['class'].values.tolist()
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi

classes, _ = get_classes()

In [33]:
classes[:2]

['none', '3']

In [34]:
print(df_test.shape, p.shape)

(2038114, 3) (2038114, 1001)


In [35]:
res = []
for i in tqdm(range(1, 1001)):
    idx = np.argsort(p[:,i])[::-1][:100000]
    best_segs = df_test.seg_id.values[idx]
    pred_str = ' '.join(best_segs)
    res.append({
        'Class': classes[i],
        'Segments': pred_str
    })



 15%|█▌        | 152/1000 [01:08<05:29,  2.58it/s]


 30%|███       | 303/1000 [02:05<04:12,  2.76it/s]


 45%|████▌     | 454/1000 [03:05<04:13,  2.15it/s]


 60%|██████    | 605/1000 [04:06<02:24,  2.74it/s]


 76%|███████▌  | 756/1000 [05:06<01:42,  2.37it/s]


 91%|█████████ | 907/1000 [06:03<00:36,  2.58it/s]


100%|██████████| 1000/1000 [06:39<00:00,  2.76it/s]

In [36]:
df_sub = pd.DataFrame(res)
df_sub.head()

,Class,Segments
0,3,1shy:15 d5yW:105 mUYZ:160 vEdm:5 mUYZ:85 F4ZL:...
1,7,BsFp:235 i2yE:50 lcGN:130 1ZYw:45 mXSB:50 d9ki...
2,8,sDmH:265 sYqm:45 sYqm:60 TJX6:65 TRB1:285 cAwR...
3,11,rpDi:260 0jGr:10 rRo2:290 kJOL:75 fqCZ:65 CJ0S...
4,12,FXxf:95 FXxf:40 FXxf:100 FXxf:90 uX9U:135 2Ase...


In [37]:
df_sub.to_csv('sub_rnn_3_8531.csv', index=False)

In [38]:
!kaggle competitions submit -c youtube8m-2019 -f sub_rnn_3_8531.csv -m 'submit'

100%|████████████████████████████████████████| 811M/811M [00:38<00:00, 22.2MB/s]
Successfully submitted to The 3rd YouTube-8M Video Understanding Challenge

In [39]:
!ls -lh

total 2.4G
-rw-rw-r-- 1 core core 6.0M Oct  6 17:34 create_meta.ipynb
-rw-rw-r-- 1 core core  65K Oct 10 10:04 eda.ipynb
-rw-rw-r-- 1 core core  75K Oct 10 18:24 submit.ipynb
-rw-rw-r-- 1 core core  13K Oct  6 17:34 submit-sigmoid.ipynb
-rw-rw-r-- 1 core core 811M Oct 10 18:26 sub_rnn_3_8531.csv
-rw-rw-r-- 1 core core 811M Oct 10 09:48 sub_transformer_3_1010_3490.csv
-rw-rw-r-- 1 core core 804M Oct 10 14:27 sub_transformer_6_gate_3934.csv
